***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-1599  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import json

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 12 21:20 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'wiki-project-unique-bucket' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.startswith("multi"):
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
parquetFile = spark.read.parquet(*paths)

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_class.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

Using the implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

**HELPING METHODS & INIT**

In [13]:
ps = PorterStemmer()
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS
def tokenize(text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [token for token in tokens if token not in all_stopwords]
    return tokens
    
def word_count(text, id, stem=False):
    tokens = tokenize(text)
    if stem:
        tokens = [ps.stem(token)for token in tokens]
    counter = {}
    for word in tokens:
        if word not in all_stopwords:
            if word not in counter:
                counter[word] = 0
            counter[word] += 1
    lst = []
    for elem in counter.keys():
        lst.append((elem, (id, counter[elem])))
    return lst

def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl)

def calculate_df(postings):
    return postings.mapValues(lambda x: len(x))

def partition_postings_and_write(postings, base_dir):
    rdd = postings.groupBy(lambda x: token2bucket_id(x[0]))
    return(rdd.map(lambda x: InvertedIndex.write_a_posting_list(x, bucket_name, base_dir)))

def calculate_docs_lengths(postings_rdd):
    return postings_rdd.flatMap(lambda x:[(curr_id,count) for curr_id,count in x[1]]).aggregateByKey(0, lambda x,y: x+y, lambda x,y: x+y)

def calculate_tf(postings_rdd):
    return postings_rdd.flatMap(lambda x: [(x[0],len(x[1]))]).aggregateByKey(0, lambda x,y: x+y, lambda x,y: x+y)

**TITLE INIT**

In [14]:
docs_titles_pairs = parquetFile.select("id","title").rdd
# Title JSON
docs_titles_pairs_dict = docs_titles_pairs.collectAsMap()
with open("titles.json","w") as titles_json:
    json.dump(docs_titles_pairs_dict, titles_json)

titles_src = "titles.json"
titles_dst = f"gs://{bucket_name}/titles/{titles_src}"
!gsutil cp $titles_src $titles_dst

Copying file://titles.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][218.2 MiB/218.2 MiB]                                                
Operation completed over 1 objects/218.2 MiB.                                    


In [15]:
# Title index
titles_counts = docs_titles_pairs.flatMap(lambda x: word_count(x[1],x[0]))
titles_postings = titles_counts.groupByKey().mapValues(reduce_word_counts)
titles_df_dict = calculate_df(titles_postings).collectAsMap()
term_total_title = calculate_tf(titles_postings)
title_dl = calculate_docs_lengths(titles_postings)
#TODO check partition_postings_and_write
_ = partition_postings_and_write(titles_postings, "titles").collect()

In [ ]:
# collect all posting lists locations into one super-set
titles_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='titles'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      titles_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = titles_posting_locs
inverted.DL = title_dl.collectAsMap()
# Add the token - df dictionary to the inverted index
inverted.df = titles_df_dict
inverted.term_total = term_total_title.collectAsMap()
# inverted.term_total = doc_text_total_terms
# write the global stats out
inverted.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/titles/{index_src}'
!gsutil cp $index_src $index_dst

**BODY INIT**

In [ ]:
doc_text_pairs = parquetFile.select("text", "id").rdd
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
term_total_body = calculate_tf(postings_filtered)
body_dl = calculate_docs_lengths(postings_filtered)
# partition posting lists and write out
# calculate total terms
_ = partition_postings_and_write(postings_filtered, "body").collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='body'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
inverted.DL = body_dl.collectAsMap()
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.term_total = term_total_body.collectAsMap()
# write the global stats out
inverted.write_index('.', 'body_index')
# upload to gs
index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/body/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

**PageRank**

Computing PageRank for the entire English Wikipedia.

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
    vertices = pages.map(lambda x: [(x[0], )] + [(y[0], ) for y in x[1]]).flatMap(lambda l: l).distinct()
    edges = pages.map(lambda y: [(y[0], z[0]) for z in y[1]]).flatMap(lambda l: l).distinct()
    return edges, vertices

In [ ]:
pages_links = spark.read.parquet(*paths).select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())

In [ ]:
pr_as_dict = {}
for item in pr.toPandas().iterrows():
    row = item[1]
    pr_as_dict[int(row[0])] = row[1]

In [ ]:
with open("pr.json", "w") as pr:
    json.dump(pr_as_dict, pr)

In [ ]:
pr_src = "pr.json"
pr_dst = f'gs://{bucket_name}/pr/{pr_src}'
!gsutil cp $pr_src $pr_dst

****Page Views****

In [ ]:
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
!wget -N $pv_path
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
    for line in f:
        parts = line.split(' ')
        wid2pv.update({int(parts[0]): int(parts[1])})

In [ ]:
with open('pv.json', 'w') as pv:
     json.dump(wid2pv, pv)

pv_src = "pv.json"
pv_dst = f'gs://{bucket_name}/pv/{pv_src}'
!gsutil cp $pv_src $pv_dst

**ANCHOR INIT**

Help Function for anchor

In [1]:
def tokenize_anchor_text(anchors):
    anchor_text = [token[1] for token in anchors]
    text = " ".join(anchor_text)
    return text

def word_count_anchor(anchor_id,anchors_text):
    return word_count(tokenize_anchor_text(anchors_text), anchor_id)
    

In [2]:
# docs_anchors_pairs = pages_links
# w_c_anchor = docs_anchors_pairs.flatMap(lambda x: word_count_anchor(x[0], x[1]))
# anchor_postings = w_c_anchor.groupByKey().mapValues(reduce_word_counts)
# anchor_postings.take(1)
# filtering postings and calculate df
w2df = calculate_df(anchor_postings)
w2df_dict = w2df.collectAsMap()
term_total_anchor = calculate_tf(anchor_postings)
anchor_dl = calculate_docs_lengths(anchor_postings)
# partition posting lists and write out
# calculate total terms
_ = partition_postings_and_write(anchor_postings, "anchor").collect()

NameError: name 'calculate_df' is not defined

In [ ]:
# collect all posting lists locations into one super-set
anchor_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='anchor'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      anchor_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = anchor_posting_locs
inverted.DL = anchor_dl.collectAsMap()
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.term_total = term_total_anchor.collectAsMap()
# write the global stats out
inverted.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/anchor/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst